In [7]:
import urllib.request as req

#Search parameters for ABRSearchByNameSimpleProtocol service
name = 'pests'
postcode = ''
legalName = ''		
tradingName = ''	
NSW = 'Y'			
SA = 'N'				
ACT = 'N'			
VIC = 'N'			
WA = 'N'				
NT = 'N'				
QLD = 'N'			
TAS = 'N'			
authenticationGuid = 'b03c103e-ab73-441a-946c-bfa45c609d63'		#Your GUID should go here

#Constructs the URL by inserting the search parameters specified above
#GETs the url (using urllib.request.urlopen)
conn = req.urlopen('https://abr.business.gov.au/abrxmlsearchRPC/AbrXmlSearch.asmx/' + 
					'ABRSearchByNameSimpleProtocol?name=' + name + 
					'&postcode=' + postcode + '&legalName=' + legalName + 
					'&tradingName=' + tradingName + '&NSW=' + NSW + 
					'&SA=' + SA + '&ACT=' + ACT + '&VIC=' +  VIC + 
					'&WA=' + WA + '&NT=' + NT + '&QLD=' + QLD + 
					'&TAS=' + TAS + '&authenticationGuid=' + authenticationGuid)
					
#XML is returned by the webservice
#Put returned xml into variable 'returnedXML' 
#Output xml string to file 'output.xml' and print to console
returnedXML = conn.read()
f = open('output.xml', 'wb')
f.write(returnedXML)
f.close
print(returnedXML)

b'<?xml version="1.0" encoding="utf-8"?>\r\n<ABRPayloadSearchResults xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns="http://abr.business.gov.au/ABRXMLSearchRPC/literalTypes">\r\n  <request>\r\n    <nameSearchRequest>\r\n      <authenticationGUID>b03c103e-ab73-441a-946c-bfa45c609d63</authenticationGUID>\r\n      <name>pests</name>\r\n      <filters>\r\n        <nameType>\r\n          <tradingName>Y</tradingName>\r\n          <legalName>Y</legalName>\r\n        </nameType>\r\n        <postcode />\r\n        <stateCode>\r\n          <QLD>N</QLD>\r\n          <NT>N</NT>\r\n          <SA>N</SA>\r\n          <WA>N</WA>\r\n          <VIC>N</VIC>\r\n          <ACT>N</ACT>\r\n          <TAS>N</TAS>\r\n          <NSW>Y</NSW>\r\n        </stateCode>\r\n      </filters>\r\n    </nameSearchRequest>\r\n  </request>\r\n  <response>\r\n    <usageStatement>The Registrar of the ABR monitors the quality of the information available on this website

In [8]:
# Final

import urllib.request as req
import xml.etree.ElementTree as ET
import pandas as pd

# Search parameters and URL construction code ...

# Parse the XML response
root = ET.fromstring(returnedXML)

# Define namespaces
ns = {'abr': 'http://abr.business.gov.au/ABRXMLSearchRPC/literalTypes'}

# Create lists to store data
abns = []
statuses = []
names = []
scores = []
indicators = []
states = []
postcodes = []

# Extract data from XML
for record in root.findall('.//abr:searchResultsRecord', ns):
    abn_element = record.find('./abr:ABN/abr:identifierValue', ns)
    abn = abn_element.text if abn_element is not None else ''
    
    status_element = record.find('./abr:ABN/abr:identifierStatus', ns)
    status = status_element.text if status_element is not None else ''
    
    # Check for organization name in different places
    name_elements = [
        record.find('./abr:mainName/abr:organisationName', ns),
        record.find('./abr:mainTradingName/abr:organisationName', ns),
        record.find('./abr:businessName/abr:organisationName', ns),
        record.find('./abr:otherTradingName/abr:organisationName', ns)  # Include otherTradingName
    ]
    name = next((elem.text for elem in name_elements if elem is not None), '')
    
    # Check for score in different places
    score_elements = [
        record.find('./abr:mainName/abr:score', ns),
        record.find('./abr:mainTradingName/abr:score', ns),
        record.find('./abr:businessName/abr:score', ns),
        record.find('./abr:otherTradingName/abr:score', ns)  # Include otherTradingName
    ]
    score = int(next((elem.text for elem in score_elements if elem is not None), 0))
    
    # Check for isCurrentIndicator in different places
    indicator_elements = [
        record.find('./abr:mainName/abr:isCurrentIndicator', ns),
        record.find('./abr:mainTradingName/abr:isCurrentIndicator', ns),
        record.find('./abr:businessName/abr:isCurrentIndicator', ns),
        record.find('./abr:otherTradingName/abr:isCurrentIndicator', ns)  # Include otherTradingName
    ]
    indicator = next((elem.text for elem in indicator_elements if elem is not None), '')
    
    state_element = record.find('./abr:mainBusinessPhysicalAddress/abr:stateCode', ns)
    state = state_element.text if state_element is not None else ''
    
    postcode_element = record.find('./abr:mainBusinessPhysicalAddress/abr:postcode', ns)
    postcode = postcode_element.text if postcode_element is not None else ''

    # Append data to lists
    abns.append(abn)
    statuses.append(status)
    names.append(name)
    scores.append(score)
    indicators.append(indicator)
    states.append(state)
    postcodes.append(postcode)

# Create a DataFrame
data = {
    'ABN': abns,
    'Identifier Status': statuses,
    'Organisation Name': names,
    'Score': scores,
    'Is Current Indicator': indicators,
    'State Code': states,
    'Postcode': postcodes
}
df = pd.DataFrame(data)

# Filter the DataFrame for accurate results
filtered_df = df[df['Score'] > 80]

# Save DataFrame to an Excel file
filtered_df.to_excel('abn_data1.xlsx', index=False)

# Display the DataFrame
print(filtered_df)

             ABN Identifier Status    Organisation Name  Score  \
0    50395891373         Cancelled            CBD Pests     97   
1    59953991576            Active            DAM PESTS     97   
2    81469969420            Active     GRAVES FOR PESTS     97   
3    18072391550         Cancelled       HANDLE'S PESTS     97   
4    89127496217         Cancelled  The Pest's Managers     97   
..           ...               ...                  ...    ...   
196  75645636166            Active         REMADE POSTS     83   
197  47608229803            Active         Reliant Pest     83   
198  14152025477         Cancelled         SCORPEX PEST     83   
199  12150414378            Active          SIMPLY PEST     83   
200  30623764349         Cancelled    STEM PEST PTY LTD     83   

    Is Current Indicator State Code Postcode  
0                      Y        NSW     2132  
1                      Y        NSW     2767  
2                      Y        NSW     2283  
3                  

In [6]:
import urllib.request as req
import xml.etree.ElementTree as ET
import pandas as pd

def query_postcodes(postcodes):
    # Initialize empty lists to store data
    all_abns = []
    all_statuses = []
    all_names = []
    all_scores = []
    all_indicators = []
    all_states = []
    all_postcodes = []

    for postcode in postcodes:
        # Search parameters and URL construction code
        name = 'pests'
        legalName = ''
        tradingName = ''
        NSW = 'Y'
        SA = 'N'
        ACT = 'N'
        VIC = 'N'
        WA = 'N'
        NT = 'N'
        QLD = 'N'
        TAS = 'N'
        authenticationGuid = 'b03c103e-ab73-441a-946c-bfa45c609d63'

        # Construct the URL by inserting the search parameters specified above
        conn = req.urlopen('https://abr.business.gov.au/abrxmlsearchRPC/AbrXmlSearch.asmx/' +
                           'ABRSearchByNameSimpleProtocol?name=' + name +
                           '&postcode=' + postcode + '&legalName=' + legalName +
                           '&tradingName=' + tradingName + '&NSW=' + NSW +
                           '&SA=' + SA + '&ACT=' + ACT + '&VIC=' + VIC +
                           '&WA=' + WA + '&NT=' + NT + '&QLD=' + QLD +
                           '&TAS=' + TAS + '&authenticationGuid=' + authenticationGuid)

        # XML is returned by the webservice
        returnedXML = conn.read()
        root = ET.fromstring(returnedXML)

        # Define namespaces
        ns = {'abr': 'http://abr.business.gov.au/ABRXMLSearchRPC/literalTypes'}

        # Extract data from XML
        for record in root.findall('.//abr:searchResultsRecord', ns):
            abn_element = record.find('./abr:ABN/abr:identifierValue', ns)
            abn = abn_element.text if abn_element is not None else ''

            status_element = record.find('./abr:ABN/abr:identifierStatus', ns)
            status = status_element.text if status_element is not None else ''

            # Check for organization name in different places
            name_elements = [
                record.find('./abr:mainName/abr:organisationName', ns),
                record.find('./abr:mainTradingName/abr:organisationName', ns),
                record.find('./abr:businessName/abr:organisationName', ns),
                record.find('./abr:otherTradingName/abr:organisationName', ns)  # Include otherTradingName
            ]
            name = next((elem.text for elem in name_elements if elem is not None), '')

            # Check for score in different places
            score_elements = [
                record.find('./abr:mainName/abr:score', ns),
                record.find('./abr:mainTradingName/abr:score', ns),
                record.find('./abr:businessName/abr:score', ns),
                record.find('./abr:otherTradingName/abr:score', ns)  # Include otherTradingName
            ]
            score = int(next((elem.text for elem in score_elements if elem is not None), 0))

            # Check for isCurrentIndicator in different places
            indicator_elements = [
                record.find('./abr:mainName/abr:isCurrentIndicator', ns),
                record.find('./abr:mainTradingName/abr:isCurrentIndicator', ns),
                record.find('./abr:businessName/abr:isCurrentIndicator', ns),
                record.find('./abr:otherTradingName/abr:isCurrentIndicator', ns)  # Include otherTradingName
            ]
            indicator = next((elem.text for elem in indicator_elements if elem is not None), '')

            state_element = record.find('./abr:mainBusinessPhysicalAddress/abr:stateCode', ns)
            state = state_element.text if state_element is not None else ''

            postcode_element = record.find('./abr:mainBusinessPhysicalAddress/abr:postcode', ns)
            postcode = postcode_element.text if postcode_element is not None else ''

            # Append data to lists
            all_abns.append(abn)
            all_statuses.append(status)
            all_names.append(name)
            all_scores.append(score)
            all_indicators.append(indicator)
            all_states.append(state)
            all_postcodes.append(postcode)

    # Create a DataFrame
    data = {
        'ABN': all_abns,
        'Identifier Status': all_statuses,
        'Organisation Name': all_names,
        'Score': all_scores,
        'Is Current Indicator': all_indicators,
        'State Code': all_states,
        'Postcode': all_postcodes
    }
    df = pd.DataFrame(data)

    # Filter the DataFrame for accurate results
    filtered_df = df[(df['Score'] > 80) & (df['Identifier Status'] == 'Active')]

    # Save DataFrame to an Excel file
    filtered_df.to_excel('abn_data.xlsx', index=False)

    return filtered_df


# List of postcodes to query
postcodes_to_query = ['2000', '2147', '2150', '2250']  # Add more postcodes as needed

# Call the function with the list of postcodes
result_df = query_postcodes(postcodes_to_query)

# Display the resulting DataFrame
print(result_df)

KeyboardInterrupt: 

In [ ]:
https://www.truelocal.com.au/search/pest-control/nsw?page=2